# Simple Workflow with pyiron_base

## Define workflow with pyiron_base

In [1]:
from pyiron_base import Project, job

In [2]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [3]:
from simple_workflow import (
    add_x_and_y as _add_x_and_y, 
    add_x_and_y_and_z as _add_x_and_y_and_z,
)

In [4]:
workflow_json_filename = "pyiron_base_simple.json"

In [5]:
add_x_and_y = job(_add_x_and_y, output_key_lst=["x", "y", "z"])
add_x_and_y_and_z = job(_add_x_and_y_and_z)

In [6]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [7]:
obj = add_x_and_y(x=1, y=2, pyiron_project=pr)

In [8]:
w = add_x_and_y_and_z(x=obj.output.x, y=obj.output.y, z=obj.output.z, pyiron_project=pr)

In [9]:
write_workflow_json(delayed_object=w, file_name=workflow_json_filename)

In [10]:
!cat {workflow_json_filename}

{"nodes": {"0": "simple_workflow.add_x_and_y_and_z", "1": "simple_workflow.add_x_and_y", "2": 1, "3": 2}, "edges": [{"target": 0, "targetHandle": "x", "source": 1, "sourceHandle": "x"}, {"target": 1, "targetHandle": "x", "source": 2, "sourceHandle": null}, {"target": 1, "targetHandle": "y", "source": 3, "sourceHandle": null}, {"target": 0, "targetHandle": "y", "source": 1, "sourceHandle": "y"}, {"target": 0, "targetHandle": "z", "source": 1, "sourceHandle": "z"}]}

## Load Workflow with aiida

In [11]:
from aiida import load_profile

load_profile()

Profile<uuid='0911f03384b24f139c7cf96149242165' name='adis'>

In [12]:
from python_workflow_definition.aiida import load_workflow_json

In [13]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [ ]:

wg.run()

03/22/2025 08:38:57 AM <1011> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [172|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/22/2025 08:38:57 AM <1011> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [172|WorkGraphEngine|continue_workgraph]: tasks ready to run: pickle_node3,pickle_node4
03/22/2025 08:38:57 AM <1011> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [172|WorkGraphEngine|run_tasks]: Run task: pickle_node3, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 1}


03/22/2025 08:38:58 AM <1011> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [172|WorkGraphEngine|run_tasks]: Run task: pickle_node4, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 2}


03/22/2025 08:38:59 AM <1011> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [172|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 180, 188
03/22/2025 08:39:01 AM <1011> aiida.scheduler.direct: [WARNING] Unrecognized job_state 'p' for job id Activating
03/22/2025 08:39:01 AM <1011> aiida.engine.transports: [ERROR] Exception whilst using transport:
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/engine/transports.py", line 106, in request_transport
    yield transport_request.future
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/engine/processes/calcjobs/manager.py", line 109, in _get_jobs_from_scheduler
    scheduler_response = scheduler.get_jobs(**kwargs)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/schedulers/plugins/direct.py", line 269, in get_jobs
    job_stats = super().get_jobs(jobs=jobs

## Load Workflow with jobflow

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
result = run_locally(flow)
result